# Example 3-6: Finding GMST and LST (Method 2)
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 191

This notebook demonstrates an alternate method for finding the Greenwich Mean Sidereal Time (GMST) and Local Sidereal Time (LST) given the UT1 date and time and  longitude.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install -r valladopy_version.txt

Then import `numpy` and the relevant `valladopy` routines:

In [2]:
import numpy as np
import valladopy.constants as const
from valladopy.mathtime.calendar import find_days
from valladopy.mathtime.utils import hms2rad

## Problem Definition
---

GIVEN: &ensp;August 20, 1992, at 12:14 P.M. UT1 at 104° west longitude<br>
FIND: &emsp;GMST ($\theta_{GMST}$) and LST ($\theta_{LST}$)

In [3]:
# Date definition
year = 1992
month = 8
day = 20
hour = 12
minute = 14
second = 0

# Longitude definition
lon = np.radians(-104)  # longitude, rad

## Solution
---

We first obtain the Greenwich sidereal time on January 1, $0^{\text{h}}$ UT1 from the *Astronomical Almanac*:

$$
\theta_{\text{GMST Jan 1,} 0^{\text{h}}} = 6^{\text{h}} \ 39^{\text{m}} \ 37.4863^{\text{s}}
$$

We can convert the hours, minutes, and seconds into radians with `hms2rad`:

In [4]:
gst0 = hms2rad(hours=6, minutes=39, seconds=37.4863)

print(f'GMST on Jan 1:\t{np.degrees(gst0):.4f}\tdeg')

GMST on Jan 1:	99.9062	deg


Next, we find the total elapsed time in solar days from the epoch January 1, $0^{\text{h}}$ UT1.

**Table 3-3** shows the numbers for the elapsed days since the start of the year, with leap-year values in parenthesis (and 1992 is a leap year):

<table>
<thead>
<tr>
<th>Date</th><th>Day of Year</th><th>Elapsed Days</th>
<th>Date</th><th>Day of Year</th><th>Elapsed Days</th>
</tr>
</thead>
<tbody>
<tr><td>January 1</td><td>1</td><td>0</td><td>July 1</td><td>182 (183)</td><td>181 (182)</td></tr>
<tr><td>January 31</td><td>31</td><td>30</td><td>July 31</td><td>212 (213)</td><td>211 (212)</td></tr>
<tr><td>February 28</td><td>59</td><td>58</td><td>August 31</td><td>243 (244)</td><td>242 (243)</td></tr>
<tr><td>March 31</td><td>90 (91)</td><td>89 (90)</td><td>September 30</td><td>273 (274)</td><td>272 (273)</td></tr>
<tr><td>April 30</td><td>120 (121)</td><td>119 (120)</td><td>October 31</td><td>304 (305)</td><td>303 (304)</td></tr>
<tr><td>May 31</td><td>151 (152)</td><td>150 (151)</td><td>November 30</td><td>334 (335)</td><td>333 (334)</td></tr>
<tr><td>June 30</td><td>181 (182)</td><td>180 (181)</td><td>December 31</td><td>365 (366)</td><td>364 (365)</td></tr>
</tbody>
</table>

Note that the day of the year is one *more* than the *elapsed days* from January 1.

We can find the *day of the year* with the `find_days` routine and subtract 1:

In [5]:
doy = find_days(year, month, day, hour, minute, second)
ed = doy - 1

print(f'Elapsed days:\t{ed:.6f} day')

Elapsed days:	232.509722 day


Now we can use **Eq. 3-49** to calculate the GMST:

$$
\theta_{\text{GMST}} = \theta_{\text{GMST} 0^{\text{h}}} + 1.002737909350795(2\pi)(ED)
$$

In [6]:
gst = np.mod(gst0 + const.RADPDAY * ed, const.TWOPI)

print(f'GMST:\t{np.degrees(gst):.4f} deg')

GMST:	152.5788 deg


Now use **Eq. 3-44** to find LST:

$$
\theta_{LST} = \theta_{GMST} + \lambda
$$

Where $\lambda$ is the observer’s longitude, positive eastward from Greenwich.

In [7]:
lst = np.mod(gst + lon, const.TWOPI)

print(f'LST:\t{np.degrees(lst):.4f} deg')

LST:	48.5788 deg


Although the computations are less complex than **Example 3-5**, notice the similarity in results, which suggests we can compare them for accuracy. As we would expect, the method using an epoch at the start of each year is very accurate towards the beginning of the year, but the errors continue to increase throughout the year. The Julian Date method is very consistent throughout the year, so we prefer it.